### Predict the employee burn out rate
https://www.hackerearth.com/ru/challenges/competitive/hackerearth-machine-learning-challenge-predict-burnout-rate/machine-learning/predict-the-employee-burn-out-rate-7-6340b4e3/

Author: Victor Krasilnikov

Score = 93,11590

Classification and Regression on Tabular Data with ktrain (https://github.com/amaiya/ktrain)

https://nbviewer.jupyter.org/github/amaiya/ktrain/blob/develop/tutorials/tutorial-08-tabular_classification_and_regression.ipynb

## Download Necessary Packages

In [1]:
import sys
#!pip install numpy
#!pip install pandas
!pip install -q ktrain

     |████████████████████████████████| 25.3MB 179kB/s 
     |████████████████████████████████| 983kB 41.1MB/s 
     |████████████████████████████████| 266kB 45.6MB/s 
     |████████████████████████████████| 1.3MB 41.2MB/s 
     |████████████████████████████████| 471kB 45.9MB/s 
     |████████████████████████████████| 2.9MB 43.0MB/s 
     |████████████████████████████████| 890kB 40.3MB/s 
     |████████████████████████████████| 1.1MB 31.2MB/s 


## Download data

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!unzip -q /content/gdrive/My\ Drive/Datasets/BURN.zip

In [4]:
!ls dataset

sample_submission.csv  test.csv  train.csv


## Import packages

In [5]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [6]:
import ktrain
from ktrain import tabular
import pandas as pd
import numpy as np
ktrain.__version__

'0.25.0'

## STEP 1: Load and Preprocess the Data

In [163]:
train=pd.read_csv('dataset/train.csv')
test=pd.read_csv('dataset/test.csv')


In [164]:
train

,Employee ID,Date of Joining,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate
0,fffe32003000360033003200,2008-09-30,Female,Service,No,2.0,3.0,3.8,0.16
1,fffe3700360033003500,2008-11-30,Male,Service,Yes,1.0,2.0,5.0,0.36
2,fffe31003300320037003900,2008-03-10,Female,Product,Yes,2.0,NaN,5.8,0.49
3,fffe32003400380032003900,2008-11-03,Male,Service,Yes,1.0,1.0,2.6,0.20
4,fffe31003900340031003600,2008-07-24,Female,Service,No,3.0,7.0,6.9,0.52
...,...,...,...,...,...,...,...,...,...
22745,fffe31003500370039003100,2008-12-30,Female,Service,No,1.0,3.0,NaN,0.41
22746,fffe33003000350031003800,2008-01-19,Female,Product,Yes,3.0,6.0,6.7,0.59
22747,fffe390032003000,2008-11-05,Male,Service,Yes,3.0,7.0,NaN,0.72
22748,fffe33003300320036003900,2008-01-10,Female,Service,No,2.0,5.0,5.9,0.52


In [165]:
train = train.dropna()
train

,Employee ID,Date of Joining,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate
0,fffe32003000360033003200,2008-09-30,Female,Service,No,2.0,3.0,3.8,0.16
1,fffe3700360033003500,2008-11-30,Male,Service,Yes,1.0,2.0,5.0,0.36
3,fffe32003400380032003900,2008-11-03,Male,Service,Yes,1.0,1.0,2.6,0.20
4,fffe31003900340031003600,2008-07-24,Female,Service,No,3.0,7.0,6.9,0.52
5,fffe3300350037003500,2008-11-26,Male,Product,Yes,2.0,4.0,3.6,0.29
...,...,...,...,...,...,...,...,...,...
22743,fffe3300390030003600,2008-12-15,Female,Product,Yes,1.0,3.0,6.0,0.48
22744,fffe32003500370033003200,2008-05-27,Male,Product,No,3.0,7.0,6.2,0.54
22746,fffe33003000350031003800,2008-01-19,Female,Product,Yes,3.0,6.0,6.7,0.59
22748,fffe33003300320036003900,2008-01-10,Female,Service,No,2.0,5.0,5.9,0.52


In [166]:
data_columns = [ 'Gender'
                , 'Company Type'
                , 'WFH Setup Available'
                , 'Designation'
                , 'Resource Allocation'
                , 'Mental Fatigue Score'
                , 'Burn Rate'
               ]

train_df = train[data_columns]


In [167]:
trn, val, preproc = tabular.tabular_from_df(train_df, label_columns=['Burn Rate'], random_state=42, is_regression=True )

processing train: 16769 rows x 7 columns
processing test: 1821 rows x 7 columns


/usr/local/lib/python3.6/dist-packages/ktrain/utils.py:580: UserWarning: Task is being treated as REGRESSION because either class_names argument was not supplied or is_regression=True. If this is incorrect, change accordingly.
  'If this is incorrect, change accordingly.')


## STEP 2: Create a Model and Wrap in Learner

In [168]:
tabular.print_tabular_regression_models()

mlp: a configurable multilayer perceptron with categorical variable embeddings [https://arxiv.org/abs/1604.06737]


In [169]:
model = tabular.tabular_regression_model('mlp', trn, metrics=[tf.keras.metrics.RootMeanSquaredError()])
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=256)

done.


## STEP 3: Train the Model

In [170]:
#learner.lr_find(max_epochs=25)
#learner.lr_plot()

In [171]:
learner.fit_onecycle(4e-3, 35)



begin training using onecycle policy with max lr of 0.004...
Epoch 1/35
66/66 [==============================] - 3s 38ms/step - loss: 0.0312 - root_mean_squared_error: 0.1767 - val_loss: 0.0061 - val_root_mean_squared_error: 0.0778
Epoch 2/35
66/66 [==============================] - 2s 37ms/step - loss: 0.0035 - root_mean_squared_error: 0.0595 - val_loss: 0.0031 - val_root_mean_squared_error: 0.0561
Epoch 3/35
66/66 [==============================] - 2s 37ms/step - loss: 0.0029 - root_mean_squared_error: 0.0537 - val_loss: 0.0030 - val_root_mean_squared_error: 0.0550
Epoch 4/35
66/66 [==============================] - 2s 37ms/step - loss: 0.0029 - root_mean_squared_error: 0.0541 - val_loss: 0.0031 - val_root_mean_squared_error: 0.0555
Epoch 5/35
66/66 [==============================] - 2s 37ms/step - loss: 0.0030 - root_mean_squared_error: 0.0545 - val_loss: 0.0031 - val_root_mean_squared_error: 0.0558
Epoch 6/35
66/66 [==============================] - 2s 37ms/step - loss: 0.0028 - 

## Step 4:  Prediction and Submission

In [172]:
test_df = test[data_columns[:-1]]

In [173]:
predictor = ktrain.get_predictor(learner.model, preproc)
preds = predictor.predict(test_df)

In [174]:
submission = pd.DataFrame(test['Employee ID'])
submission['Burn Rate'] = preds

In [175]:
submission

,Employee ID,Burn Rate
0,fffe31003300390039003000,0.613920
1,fffe31003300310037003800,0.362055
2,fffe33003400380035003900,0.426462
3,fffe3100370039003200,0.377067
4,fffe32003600390036003700,0.532387
...,...,...
12245,fffe3900310034003700,0.428519
12246,fffe32003600330034003000,0.429114
12247,fffe31003800340039003000,0.830870
12248,fffe32003600380031003800,0.577155


In [176]:

with open('KT_submission.csv', 'w') as f:
    submission.to_csv(f, index=False, header=True) 


## To download the generated csv in colab run the below command


In [177]:
try:
  from google.colab import files
  files.download('KT_submission.csv')
except ImportError as e:
  print("Only for Colab") 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>